## 英文轉西班牙文翻譯模型 - 以seq2seq Transformer 實現
### 介紹:
- 將會學到:
    1. keras layer 去向量化 text data
    2. 實現 TransformerEncoder、TransformerDecoder 以及 PositionalEmbedding layer.
    3. 準備 seq2seq data 給模型訓練
    4. 使用訓練過的模型是產生從未看過的輸入句子翻譯結果。(seq2seq inference)

In [1]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

#### 下載資料

In [2]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

2654208/2638744 [==============================] - 0s 0us/step


In [3]:
text_file

PosixPath('/root/.keras/datasets/spa-eng/spa.txt')

#### 解析資料
每一行有包含英文句子以及對應的西班牙句子。英文句子是 source sequence 而 西班牙 則是 target sequence。現在需要把 "[start]" token、"[end]" token 各加在 西班牙句子的前後頭，當作辨識符。

In [11]:
with open(text_file) as f:
    lines = f.read().split('\n')[:-1]

text_pairs = []         # (eng, spa)
for line in lines:
    eng, spa = line.split('\t')
    spa = "[start] " + spa + ' [end]'
    text_pairs.append((eng, spa))


# 看看長得如何:
for i in range(5):
    print(text_pairs[i])

('Go.', '[start] Ve. [end]')
('Go.', '[start] Vete. [end]')
('Go.', '[start] Vaya. [end]')
('Go.', '[start] Váyase. [end]')
('Hi.', '[start] Hola. [end]')


現在將資料切分成 train/validation/test

In [12]:
random.shuffle(text_pairs)      # 打亂資料，因為可以從上面明顯的看到資料有一些規律

num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples

train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples+num_val_samples]
test_pairs = text_pairs[num_train_samples+num_val_samples:]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


### 向量化 text data
將會使用 2個 TextVectorization layer 將 text data 向量化(1 for eng, 1 for spa)，也就是說把原始的文字資料轉化成 int seqs，其每一個int代表 index of a word in vocabulary。

英文的用預測的標準清除即可，透過空白分開；而spa 需要客製化，將標點符號去除清單加入"¿"。(spa 特有)。

但在真實的應用中，我們可能不選擇去除標點符號。

In [15]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", '')
strip_chars = strip_chars.replace("]", '')

vocab_size = 15000
sequence_length = 20
batch_size = 64

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, '[%s]' % re.escape(strip_chars), "")

# vocab_size 都用一樣只是剛好，可以各自設定。
eng_vectorization = TextVectorization(
    max_tokens=vocab_size, 
    output_mode='int',
    output_sequence_length=sequence_length
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length + 1, # why?
    standardize=custom_standardization
)

train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
# fit 差不多的意思
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

接下來，將會將其組成 datasets.

每一個訓練步，模型將會找到 predict target word N+1 (持續) 透過使用 source sentence and the target words 0-N(前面已確認的word)。

因此，訓練資料集將會不斷產生 (inputs, targets):
- inputs: 
    - 是一個python dict，key="encoder_inputs";key="decoder_inputs"，可以說使用0-N的word去預測N+1的words
- target: 
    - target sentence offset by one step，提供了下一個將要預測的words in the target sentence

In [26]:
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return ({
        "encoder_inputs": eng,
        "decoder_inputs": spa[:, :-1] # 0-N的概念
        }
        ,spa[:, 1:]  
    )

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)       # 處理成dict
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

快速看一下資料集是否正確:


In [28]:
for inputs, targets in train_ds.take(1):
    print(f"inputs[encoder_inputs].shape: {inputs['encoder_inputs'].shape}")
    print(f"inputs[decoder_inputs].shape: {inputs['decoder_inputs'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs[encoder_inputs].shape: (64, 20)
inputs[decoder_inputs].shape: (64, 20)
targets.shape: (64, 20)


### 建立模型
我們的 seq2seq Transformer 模型包含了 Encoder、Decoder 部分，要讓模型意識到句子的順序訊息則透過PositionalEmbedding layer，詳細可以看 Attention is all you need 論文。

source sentence 將會 傳入 Encoder，其將會產生一個新的 representation of it. 這個新的 representation 將會傳給 Decoder，與 target sequence (0-N)一起。 Decoder 將會去預測 **下一個** words in the target sequence (N+1 and beyond).

一個關鍵讓上述可以成功地點在於，我們用了 **causal masking**。Decoder 會一次看整個句子，我們必須確保 Decoder 只利用了 0-N 的 target tokens 去預測 N+，不然將會用到我們不該看到的tokens。

In [33]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim,
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation='relu'), 
             layers.Dense(embed_dim)]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True
    
    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis,], dtype='int32')    # 配合attention, 但我對於維度沒有到非常清楚。
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask,
        )
        # short-cut
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length,
            output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_sie = vocab_size
        self.embed_dim = embed_dim
    
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True
    
    def get_causal_attention_mask(self, inputs):
        # 要詳細看看
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype='int32')
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0
        )
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

組成 end-to-end model

In [34]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

### 訓練模型
我們將會使用 accuracy 作為一個快速判斷的 metrics，去監控訓練、驗證成效。真正在翻譯任何上比較常去看BLEU scores 作為 metrics。

因為訓練時長關係，先用epochs=1來繼續。

In [38]:
epochs = 10  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding_2 (Positi  (None, None, 256)   3845120     ['encoder_inputs[0][0]']         
 onalEmbedding)                                                                                   
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   3155456     ['positional_embedding_

### Decoding test sentences
終於到了最後階段，掩飾如何去把新輸入的英文語句去翻譯。我們將會把向量化的英語句子以及 target token "[start]" 然後不斷產生下一個句子，直到我們遇到 "[end]" 才停止。

In [39]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print(input_sentence, '--->', translated)

I didn't know you couldn't drive. ---> [start] no sabía que no podría conducir [end]
Tom doesn't like to be kept waiting for a long time. ---> [start] a tom no le gusta estar esperando un rato [end]
Does your father get home early? ---> [start] tu padre se [UNK] pronto [end]
Bring food. ---> [start] [UNK] comida [end]
I often catch colds. ---> [start] me tengo [UNK] [end]
Tom frequently speaks with Mary. ---> [start] tom casi no habla con mary [end]
Don't lie to Tom. ---> [start] no te [UNK] a tom [end]
Tom works in a factory. ---> [start] tom trabaja en una fábrica [end]
Tom won't answer. ---> [start] tom no va a responder [end]
I frequently think about my mother who passed away. ---> [start] yo no creo que mi madre me ha estado lejos de mi madre [end]
Tom suddenly felt like he was going to faint. ---> [start] tom se sintió que estaba [UNK] a [UNK] [end]
Everything was exciting to me when I visited Spain for the first time. ---> [start] me [UNK] me fue cuando [UNK] la vez que [UNK] po